In [1]:
%pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\diyar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [3]:
from langchain.document_loaders import DirectoryLoader

In [5]:
# You MUST add your PDF to local files in this notebook (folder icon on left hand side of screen)

# Simple method - Split by pages 
root_dir = "C:\\Users\\diyar\\Desktop\\bil496\\webapp\\romePdf"
loader = DirectoryLoader(f'{root_dir}', glob="./*.pdf", loader_cls=PyPDFLoader)
pages = loader.load_and_split()
print(pages[0])


page_content='ℹ ℹPDF\nWhen folding the\nsheet, we would\nsuggest placing  \npressure with your\nfingers on the side to\nbe folded,\nso that it might open\nup,\nbut if you want to\npermanently remedy\nthis problem, \nit is enough to apply a\nvery small amount of\nglue.Starting with the printer set-up:\nselect A4 format \nand change \nthe direction of the paper \nfrom vertical to horizontal.\nWe can start to\nprint your guide,\nin the new and fast pdf format\nNow you will have \nprinted the whole document                  \n   Fold the sheet exactly in the\ncentre, along an imaginary line,keeping the printed side to theoutside,repeat this operation for all pages.\nNow you will have a mountain of\nflapping sheets in front of you, do not worry, we are almostthere, the only thing left to do, is to re-bind the whole guide by the edges of thelongest sides of the sheets, with  a normalstapler (1) or, for a moreaesthetic result, referring the work to abookbinder asking for spiralbinding(2). \nC

In [35]:
pages[0]

Document(page_content='ℹ ℹPDF\nWhen folding the\nsheet, we would\nsuggest placing  \npressure with your\nfingers on the side to\nbe folded,\nso that it might open\nup,\nbut if you want to\npermanently remedy\nthis problem, \nit is enough to apply a\nvery small amount of\nglue.Starting with the printer set-up:\nselect A4 format \nand change \nthe direction of the paper \nfrom vertical to horizontal.\nWe can start to\nprint your guide,\nin the new and fast pdf format\nNow you will have \nprinted the whole document                  \n   Fold the sheet exactly in the\ncentre, along an imaginary line,keeping the printed side to theoutside,repeat this operation for all pages.\nNow you will have a mountain of\nflapping sheets in front of you, do not worry, we are almostthere, the only thing left to do, is to re-bind the whole guide by the edges of thelongest sides of the sheets, with  a normalstapler (1) or, for a moreaesthetic result, referring the work to abookbinder asking for spiralbindin

In [36]:
def split_page_into_two(page_content):
    # This is a simple split function, you might need to customize it
    # based on the structure of your documents.
    midpoint = len(page_content) // 2
    return page_content[:midpoint], page_content[midpoint:]

def create_documents_from_pages(pages):
    documents = []
    for page in pages:
        part1, part2 = split_page_into_two(page)
        documents.append(part1)
        documents.append(part2)
    return documents

nembeddings = OpenAIEmbeddings()
# Load and split pages
npages = loader.load_and_split()

# Create documents from pages
ndocuments = create_documents_from_pages(npages)

# Use these documents for creating embeddings and FAISS index
ndb = FAISS.from_documents(ndocuments, nembeddings)

TypeError: object of type 'Document' has no len()

In [ ]:
npages[0]

In [ ]:
nquery = "nightlife?"
ndocs = ndb.similarity_search(nquery)
ndocs[1]

In [19]:
# Get embedding model
embeddings = OpenAIEmbeddings()

# Create vector database
db = FAISS.from_documents(pages, embeddings)

In [30]:
query = "nightlife?"
docs = db.similarity_search(query)
docs[1]

Document(page_content='THE ROME CITY  GUIDE 15\n© Netplan - Internet solutions for tourismTHE ROME CITY  GUIDE 16\n© Netplan - Internet solutions for tourismItalian and international artists all appear in \nconcert.\n \n Sana a Roma , April\nThis Mediterranean Trade Fair for Natural \nProducts has been organized for a few \nyears now at the Rome Trade Fair District. \nExhibitions of bio-products, conferences and \ntasting.\n \nEstate Romana , from June to September\nThis includes all kinds of events, from \nmusic to theater, literary meetings and \ncinema. Events that take place in the most \ncharacteristic places in Rome that attract the \nparticipation of thousands of artists from all \nover the world.\n \nDonna Sotto le Stelle , July\nThe annual Roman appointment with high \nfashion. Creations by the most famous \nfashion designers are modeled on a catwalk \nin a truly magnificent setting: on the Spanish \nSteps in Piazza di Spagna .\nMany international musical events are \norganize

In [34]:
docs[1]

Document(page_content='THE ROME CITY  GUIDE 15\n© Netplan - Internet solutions for tourismTHE ROME CITY  GUIDE 16\n© Netplan - Internet solutions for tourismItalian and international artists all appear in \nconcert.\n \n Sana a Roma , April\nThis Mediterranean Trade Fair for Natural \nProducts has been organized for a few \nyears now at the Rome Trade Fair District. \nExhibitions of bio-products, conferences and \ntasting.\n \nEstate Romana , from June to September\nThis includes all kinds of events, from \nmusic to theater, literary meetings and \ncinema. Events that take place in the most \ncharacteristic places in Rome that attract the \nparticipation of thousands of artists from all \nover the world.\n \nDonna Sotto le Stelle , July\nThe annual Roman appointment with high \nfashion. Creations by the most famous \nfashion designers are modeled on a catwalk \nin a truly magnificent setting: on the Spanish \nSteps in Piazza di Spagna .\nMany international musical events are \norganize